In [9]:
from data import data_dict, DIRECTORY
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from nltk.tokenize import word_tokenize
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import nltk
import torch
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from keras.preprocessing import sequence
import keras
from keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

data_dict = {'fake': 'data/fake.csv', 'political_social_media': 'data/political_social_media.csv', 'politifact': 'data/truth-detectiondeception-detectionlie-detection/politifact.csv', 'news_articles': 'data/news_articles.csv', 'politifact_clean': 'data/truth-detectiondeception-detectionlie-detection/politifact_clean.csv', 'politifact_clean_binarized': 'data/truth-detectiondeception-detectionlie-detection/politifact_clean_binarized.csv'}

class PreprocessingDataset(Dataset):
    def __init__(self, file, root, x_col, y_col, meta_columns, label_idx = -1):
        self.x_col = x_col
        self.y_col = y_col
        self.data = pd.read_csv(file)
        self.data = self.data.sample(frac=1).reset_index(drop=True)
        self.data = self.data.drop(meta_columns, axis=1)

        # self.data, self.base_ref = self.tokenizer(self.data, [x_col])
        self.x_data = self.data[x_col]
        self.max_len = max([len(i) for i in self.x_data])

        self.x_data = self.word_vector(self.x_data)
        self.data[x_col] = [torch.LongTensor(i) for i in self.x_data]
        self.data = self.vectorize(self.data, [y_col])
        self.df_data = self.data
        self.data = self.data.to_numpy()

        self.root = root
        self.transform = transforms.Compose([transforms.ToTensor()])

    def format_text(self, token):
        clean_token = ''.join(chr for chr in token if chr.isalnum() and chr.isalpha())
        return clean_token

    def word_vector(self, data):
        x_data = data
        x_data = list(x_data)
        maximum_length = 0
        max_idx = 0
        for idx, i in enumerate(x_data):

            if len(i) > maximum_length:
                maximum_length = len(i)
                max_idx = idx
        
        t = Tokenizer()
        t.fit_on_texts(x_data)
        sequences = t.texts_to_sequences(x_data)
        sequences = keras.preprocessing.sequence.pad_sequences(sequences, maxlen=maximum_length)
        print(x_data[0])
        print(len(x_data[0]))
        print(sequences[0])

        return sequences


    def vectorize(self, data_inp, columns):
        data = data_inp
        for column in columns:
            labels = list(data[column].unique())
            ref = dict(zip(data[column].unique(), [i for i in range(len(labels))]))
            print(ref)
            for idx, val in enumerate(data[column]):
                vectorized = ref[data[column][idx]]
                data[column][idx] = torch.tensor(vectorized, dtype=float)
        return data

    def __len__ (self):
        return len(self.data)

    def __getitem__ (self, idx):
        
        self.transpose_data = self.data
        self.transpose_data = self.transpose_data.transpose()
        x_data = self.transpose_data[0]
        y_data = self.transpose_data[1]

        return x_data[idx], y_data[idx]

print(data_dict)
p = '/Users/devpatelio/Downloads/Coding/Global_Politics_EA/data/truth-detectiondeception-detectionlie-detection/politifact_clean_binarized.csv'
clean_truth_data = PreprocessingDataset(p, DIRECTORY, 'statement', 'veracity', ['source', 'link'])

{'fake': 'data/fake.csv', 'political_social_media': 'data/political_social_media.csv', 'politifact': 'data/truth-detectiondeception-detectionlie-detection/politifact.csv', 'news_articles': 'data/news_articles.csv', 'politifact_clean': 'data/truth-detectiondeception-detectionlie-detection/politifact_clean.csv', 'politifact_clean_binarized': 'data/truth-detectiondeception-detectionlie-detection/politifact_clean_binarized.csv'}
Says Barack Obama "founded ISIS. I would say the co-founder would be crooked Hillary Clinton."
94
[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0  

<ipython-input-9-62f58ebcb533>:76: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[column][idx] = torch.tensor(vectorized, dtype=float)


In [10]:
clean_truth_data.df_data.head()

,statement,veracity
0,"[tensor(0), tensor(0), tensor(0), tensor(0), t...",0
1,"[tensor(0), tensor(0), tensor(0), tensor(0), t...",0
2,"[tensor(0), tensor(0), tensor(0), tensor(0), t...",0
3,"[tensor(0), tensor(0), tensor(0), tensor(0), t...",1
4,"[tensor(0), tensor(0), tensor(0), tensor(0), t...",1


In [11]:
BATCH_SIZE = 64

primary_data = clean_truth_data #secondary option of truth_data

train_len = int(len(primary_data)*0.8)
test_len = len(primary_data) - train_len

train_set, test_set = torch.utils.data.random_split(primary_data, [train_len, test_len])

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=True)

print(len(train_set))
print(len(test_set))

num_feats = np.array([train_set[i][0]for i in range(len(train_set))])
num_labels = np.array([train_set[i][1]for i in range(len(train_set))])

a = iter(train_loader)
b = next(a)
b = np.asarray(b)
print(b.shape)
inp_size = (b[0].shape)[1]
print(inp_size)

8950
2238
(2,)
400


<ipython-input-11-9e81bf2c5f3d>:16: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  num_feats = np.array([train_set[i][0]for i in range(len(train_set))])
<ipython-input-11-9e81bf2c5f3d>:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  num_feats = np.array([train_set[i][0]for i in range(len(train_set))])
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/

In [12]:
import itertools
ab = list(itertools.chain(*[i[0] for i in clean_truth_data]))
print(len(ab))
ab = set([int(i) for i in ab])
emb_dim = len(ab)

4475200


In [15]:
import torch.nn as nn
import torch.nn.functional as F

class RecurrentClassifier(nn.Module):
    def __init__(self, embedding_dim, input_size, hidden_size, output_size, num_layers, dropout=0.3):
        super(RecurrentClassifier, self).__init__()

        self.embedding = nn.Embedding(embedding_dim, input_size)
        self.rnn = nn.LSTM(input_size, 
                            hidden_size,
                            num_layers,
                            batch_first = True,
                            dropout=dropout)
        self.fc1 = nn.Linear(hidden_size*2, hidden_size)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.embedding(x)
        x, (hidden, cell) = self.rnn(x)
        print(hidden.shape)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1, :, :]), dim=1))
        x = self.fc1(hidden)
        x = self.dropout(self.fc2(x))
        x = torch.squeeze(x)

        return x


max_len = len(train_set[1][0])
ref_check = 1
print(max_len)

net = RecurrentClassifier(emb_dim, int(inp_size), 50, ref_check, 2, dropout=0.2)
print(net)

400
RecurrentClassifier(
  (embedding): Embedding(13784, 400)
  (rnn): LSTM(400, 50, num_layers=2, batch_first=True, dropout=0.2)
  (fc1): Linear(in_features=100, out_features=50, bias=True)
  (fc2): Linear(in_features=50, out_features=1, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [14]:

LR = 1e-3
optimizer = torch.optim.Adam(net.parameters(), lr=LR, weight_decay=5e-3)
loss_func = torch.nn.BCEWithLogitsLoss()

epochs = 1000
losses = []

for step in range(epochs):
    running_loss = 0.0
    for i, data in enumerate(train_loader):
        inp, labels = data
        inp, labels = inp.long(), labels.float()
        optimizer.zero_grad()
        outputs = net(inp)
        cost = loss_func(outputs, labels)
        cost.backward()
        optimizer.step()

        running_loss += cost.item()
    print(f'Epoch: {step}   Training Loss: {running_loss/len(train_loader)}')
print('Training Complete')  

KeyboardInterrupt: 

In [ ]:
print(data_dict)

pd.read_csv(data_dict['politifact_clean_binarized']).head()

{'fake': 'data/fake.csv', 'political_social_media': 'data/political_social_media.csv', 'politifact': 'data/truth-detectiondeception-detectionlie-detection/politifact.csv', 'news_articles': 'data/news_articles.csv', 'politifact_clean': 'data/truth-detectiondeception-detectionlie-detection/politifact_clean.csv', 'politifact_clean_binarized': 'data/truth-detectiondeception-detectionlie-detection/politifact_clean_binarized.csv'}


,statement,source,link,veracity
0,"Sen. Kamala Harris is ""supporting the animals ...",Donald Trump,/web/20180705082623/https://www.politifact.com...,0
1,"Says Ronald Reagan said immigrants ""brought wi...",Becoming American Initiative,/web/20180705082623/https://www.politifact.com...,1
2,"Says Democratic Senators ""demand Supreme Court...",Viral image,/web/20180705082623/https://www.politifact.com...,0
3,"""Tim Kaine doesn’t want a border at all. He wa...",Corey Stewart,/web/20180705082623/https://www.politifact.com...,0
4,"""George H.W. Bush has died at 94.""",Bloggers,/web/20180705082623/https://www.politifact.com...,0
